In [1]:
from inforetlib.data import *
from inforetlib.synonyms import *

In [2]:
ld = LoadData()

Loading topics

In [3]:
topicn,query = ld.load_topics()

Loading Corpus

In [4]:
corpus = ld.load_corpus()


Loading query results

In [5]:
qtopicn,idn,label = ld.load_query_results()

for each query, we pass to the ir engine, which scans the corpus and returns query results. We then compare these query results against the query results data.

because of the way the algorithm works, we need to collect synonyms up to num_synonyms 

We also need to normalize the resulting figure to take into account that some words will have many synonyms or big clusters, and some will have few. The scores for the ones with few synonyms, and long documents, will necessarily be lower, but that doesn't necessarily mean they are less relevant. Thus there ought to be a normalization factor which accounts for the number of synonyms/size of cluster and also the length of the document.

We also need to have a maximum number of synonyms/clustered selected words as a parameter.

In [6]:
syn = Synonyms()

In [9]:
syn.get('intelligence',stem=False)

['acumen',
 'agility',
 'brilliance',
 'intellect',
 'judgment',
 'perception',
 'quickness',
 'savvy',
 'sense',
 'skill',
 'subtlety',
 'understanding',
 'wit',
 'IQ',
 'acuity',
 'alertness',
 'aptitude',
 'brainpower',
 'brains',
 'brightness',
 'capacity',
 'cleverness',
 'comprehension',
 'coruscation',
 'discernment',
 'luminosity',
 'mentality',
 'mind',
 'penetration',
 'perspicacity',
 'precocity',
 'quotient',
 'reason',
 'sagacity',
 'smarts',
 'trenchancy']

In [10]:
class SynonymRetrieval:
    def __init__(self):
        self.__syn = Synonyms()

    



In [14]:
ex = corpus[0]['text']

In [36]:
def preprocess_document(string):
    string = string.replace('-',' ')
    string = ''.join([i for i in string if (i.isalpha()) or (i == ' ')])
    string = string.lower()
    string = nltk.word_tokenize(string)
    string = list(set(string))
    # remove stopwords
    string = [word for word in string if not word in nltk.corpus.stopwords.words()]
    ps = nltk.stem.PorterStemmer()
    string = [ps.stem(w) for w in string]
    # stem
    return set(string)

In [ ]:
def preprocess_query(query):
    # need to get synonyms

    # need to remove stopwords from query

    # need to tokenize and stem synonyms and query

    # need to return each as a set

Get set intersection. Discount factor determines the divisor for the jaccard score of the either synonyms, or clustered words

In [37]:
ex = preprocess(ex)

{'chang', 'coronaviru', 'respons', 'weather'}